This is a link budget for a simple one directional free space optical link

# Imports

In [13]:
!pip install raypier
!pip install pyoptools
!pip install pythreejs


from google.colab import output
output.enable_custom_widget_manager()

import numpy as np
import scipy.constants as constants
from pyoptools.all import *
import time

import raypier

  Using cached raypier-0.2.3.tar.gz (1.4 MB)
  Preparing metadata (setup.py) ... done
  Using cached numpy-1.19.0.zip (7.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (pyproject.toml) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


ModuleNotFoundError: No module named 'raypier'

# Transmit

Possible choice for source laser:
Rohm single mode 220mW NIR laser. Available in 820nm, 840nm and 850nm.

[Rohm 850nm laser](https://fscdn.rohm.com/en/products/databook/datasheet/opto/laser_diode/infrared/rld85nzj400a008-e.pdf)

In [ ]:
tx_power_W = 0.22 #@param {"type":"number"}
tx_wavelength_m = 850e-9 #@param {"type":"number"}
tx_fast_axis_divergence_deg = 9.5 #@param {"type":"number"}
tx_slow_axis_divergence_deg = 19 #@param {"type":"number"}


In [11]:

def laser_source(
    wavelength_m,
    power_w,
    num_rays,
    div_deg,
    label="",
    draw_color=None):
  """ Ray bundle for a single mode laser.

  The emitting area is simulated as a point source. It should be modeled as a
  gaussian beam waist, but because we are going to defocus the beam on purpose,
  it should not make too much of a difference.

  Parameters:
    wavelength_m: The source wavelength, in meters,
    power_w: The source total power, in watts,
    num_rays: Tuple (nx,ny) with the number of rays used to create the beam.
    div_deg: Tuple with the beam's horizontal and vertical divergence.
    label: String used to identify the ray source
    draw_color: Color used to represent the rays in plots. Default (None)
                automatically chooses a color based on the wavelength.
                Otherwise, can be any valid matplotlib color descriptor. See :
                https://matplotlib.org/3.3.1/api/colors_api.html

  Returns:
    A list of ray objects.
  """
  nx, ny = num_rays
  dx, dy = div_deg

  # We are going to sample the beam uniformly, and assign a varying power to
  # each ray. This will allow us to better simulate what happens at the edge
  # of the lens



#   ret_val = []

#    nx, ny = num_rays
#    dx, dy = size

#    # note modify this to use traits
#    dx = float(dx)
#    dy = float(dy)

#    for ix in range(nx):
#        for iy in range(ny):
#            x = -dx / 2.0 + dx * ix / (nx - 1)
#            y = -dy / 2.0 + dy * iy / (ny - 1)
#            ret_val.append(
#                Ray(
#                    origin=(x, y, 0),
#                    direction=(0, 0, 1),
#                    wavelength=wavelength,
#                    label=label,
#                    draw_color=draw_color,
#                ).ch_coord_sys_inv(origin, direction)
#            )
#    return ret_val


L=library.Edmund["32475"]
SEN=CCD(size=(20,20))#c.rotateZ(D[2])
    #c.rotateY(D[1])
    #c.rotateX(D[0])

R=parallel_beam_c(origin=(0.0, 0.0, 0.0), direction=(0.0, 0.0, 0.0),
                  size=(10, 10), num_rays=(200, 200), wavelength=0.58929)
S=System(complist=[(L,(0,0,50),(0,0,0)),(SEN,(0,0,150),(0,0,0))],n=1)
tic = time.time()
#S._np_rays += R
S.ray_add(R)
toc = time.time()
print(toc - tic)
tic = time.time()
S.propagate()
toc = time.time()
print(toc - tic)
#renderer = Plot3D(S)
#display(renderer)



0.0014452934265136719
7.8498547077178955


# Optical budget
Both TX and RX are simple collimators with a limited divergent beam. We assume that the beams are both uniform in angular distribution and spatially, sampled at the waist.

In [3]:
# Parameters
link_distance_m = 1000 #@param {"type":"number"}
rx_aperture_diameter_m = 0.05 #@param {"type":"number"}
rx_f_number = 1 #@param {"type":"number"}
rx_optical_transmission = 0.8 #@param {"type":"number"}
rx_apd_diameter_m = 500e-9 # @param {"type":"number"}
tx_divergence_rad = 10e-3 #@param {"type":"number"}
tx_optical_transmission = 0.8 #@param {"type":"number"}
tx_aperture_diameter_m = 0.05 #@param {"type":"number"}

# Beam divergences
rx_focal_length_m = rx_aperture_diameter_m * rx_f_number
rx_divergence_rad = rx_apd_diameter_m / rx_focal_length_m
tx_divergence_deg = np.degrees(tx_divergence_rad)
rx_divergence_deg = np.degrees(rx_divergence_rad)

# Aperture surface area
rx_aperture_area_m2 = np.pi * (rx_aperture_diameter_m / 2)**2
tx_aperture_area_m2 = np.pi * (tx_aperture_diameter_m / 2)**2
# Beam footprint at full the receiver
rx_beam_diameter_m = link_distance_m * tx_divergence_rad
rx_beam_area_m2 = np.pi * (rx_beam_diameter_m / 2)**2
# Full transmission
link_transmission = tx_optical_transmission * rx_optical_transmission * rx_aperture_area_m2 / rx_beam_area_m2
rx_power_W = tx_power_W * link_transmission
# Link loss (dBm)
link_loss_dBm = 10 * np.log10(link_transmission)

# Display results
print("TX")
print(f"Power: {tx_power_W:.2g} W")
print(f"Wavelength: {tx_wavelength_m * 1e9} nm")
print(f"Beam divergence: {tx_divergence_rad * 1e3:.2f}mrad (+/-{tx_divergence_deg/2:.2f} deg)")
print("")

print("RX")
print(f"Beam divergence: {rx_divergence_rad * 1e3:.2f}mrad (+/-{rx_divergence_deg/2:.2f} deg)")
print(f"Beam diameter: {rx_beam_diameter_m:.2f} m")
print(f"Power on detector: {rx_power_W:.2g} W")
print("")

print("Link")
print(f"Link loss: {link_loss_dBm:.2f} dBm")



TX
Power: 0.22 W
Wavelength: 850.0 nm
Beam divergence: 10.00mrad (+/-0.29 deg)

RX
Beam divergence: 0.01mrad (+/-0.00 deg)
Beam diameter: 10.00 m
Power on detector: 3.5e-06 W

Link
Link loss: -47.96 dBm


In [4]:

# Link parameters
link_rate_bps = 3e9 #@param
link_spectral_efficiency = 2 #@param

# Transmit parameters

# Receive parameters
rx_apd_capacitance_F = 1.2e-12 #@param
rx_apd_gain_ApW = 65 #@param
rx_tia_input_impedance_ohm = 50 #@param
rx_tia_transimpedance_ohm = 1000 #@param


# RX Signal
- The receiver is composed of a silicon APD, followed by a TIA
- We assume that the TIA has more bandwidth than the APD. The bandwidth limiting factor is the first order low pass formed by the APD capacitance and TIA input impedance.


In [5]:
# Number of photons per bit
photon_energy_J = constants.h * constants.c / tx_wavelength_m
num_photons_per_bit = rx_power_W / (photon_energy_J * link_rate_bps)
# Signal amplitude (assuming DC balance)
rx_amplitude_W = rx_power_W / 2
rx_amplitude_A = rx_power_W * rx_apd_gain_ApW
rx_amplitude_V = rx_amplitude_A * rx_tia_transimpedance_ohm

# Input bandwidth
rx_bandwidth_Hz = 1 / (rx_tia_input_impedance_ohm * rx_apd_capacitance_F * 2 * np.pi)
rx_required_spectral_efficiency = link_rate_bps / rx_bandwidth_Hz

# Display results
print(f"Power on detector: {rx_power_W:.2g} W")
print(f"Number of photons per bit: {num_photons_per_bit:.2f}")
print(f"Minimal required spectral efficiency (NRZ OOK is 2): {rx_required_spectral_efficiency:.2f}")
print(f"Signal amplitude: {rx_amplitude_V:.2f} V")
print(f"APD Bandwidth: {rx_bandwidth_Hz / 1e9:.2f} GHz")



Power on detector: 3.5e-06 W
Number of photons per bit: 5020.69
Minimal required spectral efficiency (NRZ OOK is 2): 1.13
Signal amplitude: 0.23 V
APD Bandwidth: 2.65 GHz


# Eye safety
- We use IEC 60825.1:2014, which has been adopted by the FDA as regulating eye safety standard in the US. A copy of the doc can be purchased directly from the IEC, or the Australian version (which is identical to the US version) can be downloaded directly from [here](https://research.unsw.edu.au/document/1-2014%20Safety%20of%20laser%20products_Equipment%20classification%20and%20requirements.pdf)
- To simplify the analysis we assume that the beam has the worst possible angular subtense (it will focus perfectly on in one spot on the retina). Additional margin could be extracted by considering the fact that the beam is actually divergent (factor C6 will not be 1).
- We assume a wavelength range of 700nm to 1050nm. AEL formulas will be different for wavelengths outside of this range.
- We assume a uniform beam. If the TX beam is non uniform, power will be higher through the limiting aperture !
- Relevant table sin IEC 60825:
 - Table 3: AEL formulas
 - Table 9: Values for parameters C4 and C7
 - Table A.6: Limiting aperture size
- We assume that the signal is 8b/10b encoded, so average power is 1/2 of peak power and that the maximum run length is 5.


In [6]:
# Compute the AEL.
# C4 (see table 9)
eye_c4 = 10**(0.002 * (tx_wavelength_m * 1e9 - 700))
# C7 (see table 9)
eye_c7 = 1
# AEL for CW conditions (see table 3)
eye_ael_cw_w = 3.9e-4 * eye_c4 * eye_c7
# AEL for each bit (1 pulse)
eye_ael_pulse_j = 7.7e-8 * eye_c4
# Limiting aperture (see table A.6)
eye_aperture_diameter_m = 0.007
eye_aperture_area_m2 = (eye_aperture_diameter_m / 2)**2 * np.pi
# TX peak power through limiting aperture (assuming uniform TX beam)
max_pulse_length = 5
duty_cycle = 0.5
eye_aperture_power_W = duty_cycle * tx_power_W * eye_aperture_area_m2 / tx_aperture_area_m2
eye_aperture_pulse_energy_j = eye_aperture_power_W * max_pulse_length / link_rate_bps

# Display results
print(f"Limiting aperture diameter: {eye_aperture_diameter_m * 1e3:.1f} mm")
print(f"Limiting aperture area: {eye_aperture_area_m2 * 1e6:.2f} mm^2")
print(f"AEL for CW signal: {eye_ael_cw_w * 1e3:.2f} mW")
print(f"TX CW power through limiting aperture ({duty_cycle * 100:.0f}% duty cycle): {eye_aperture_power_W * 1e3:.2f} mW")
print(f"AEL for pulse: {eye_ael_pulse_j * 1e9:.2f} nJ")
print(f"TX max pulse ({max_pulse_length} bits) energy through limiting aperture: {eye_aperture_pulse_energy_j * 1e9:.2g} nJ")


Limiting aperture diameter: 7.0 mm
Limiting aperture area: 38.48 mm^2
AEL for CW signal: 0.78 mW
TX CW power through limiting aperture (50% duty cycle): 2.16 mW
AEL for pulse: 153.64 nJ
TX max pulse (5 bits) energy through limiting aperture: 0.0036 nJ
